# Robust Identification of Investor Beliefs

by [Xiaohong Chen](https://economics.yale.edu/people/faculty/xiaohong-chen), [Lars Peter Hansen](http://larspeterhansen.org/) and [Peter G. Hansen](https://mitsloan.mit.edu/phd/students/peter-hansen).

The latest draft of the paper can be found [here](http://larspeterhansen.org/research/papers/).

Notebook by: Han Xu, Zhenhuan Xie

## 1. Overview

This notebook provides source code and explanations for how we solve the dynamic problem in Section 5. It also provides source code for the table and plots in Section 6. 

## 2. Intertemporal Divergence

Recall **Problem 5.5** in the paper:

\begin{equation}
\mu = \min_{N_1\in \mathcal{N}} \mathbb{E}\left(N_1\left[g(X_1)+\xi\log N_1 + v_1\right]\mid \mathfrak{I}_0\right) - v_0
\end{equation}
*subject to constraints*:
\begin{equation}
\mathbb{E}\left[N_1 f(X_1)\mid\mathfrak{I}_0\right] = 0
\end{equation}

By **Proposition 5.7**, this problem can be solved by finding the solution to:

\begin{equation}
\epsilon = \min_{\lambda_0}\mathbb E \left(\exp \left[-\frac{1}{\xi}g(X_1)+\lambda_0\cdot f(X_1)\right]\left( \frac{e_1}{e_0}\right) \mid \mathfrak{I}_0\right)
\end{equation}

*where*
\begin{align*}
\mu &= -\xi \log \epsilon,\\
v_0 &= -\xi \log e_0.
\end{align*}

The implied solution for the probablity distortion is:

\begin{equation}
N_1^* = \frac{\exp \left[-\frac{1}{\xi}g(X_1)+\lambda^*_0(Z_0)\cdot f(X_1)\right]e_1^*}{\epsilon^*e_0^*}
\end{equation}

where $\lambda^*_0$ is the optimizing choice for $\lambda_0$ and $\left(\epsilon^*,e_0^*\right)$ are selected so that the resulting $\sf Q$ induces stochastically stable. The conditional expectation implied by the bound is

\begin{equation}
\mathbb{E}\left[N_1^*g(X_1)\mid \mathfrak{I}_0\right]
\end{equation}

which in turn implies a bound on the unconditional expectation equal to

\begin{equation}
\int \mathbb{E}\left[N_1^*g(X_1)\mid\mathfrak{I}_0\right]d \sf Q_0^*
\end{equation}

The implied relative entropy is

\begin{equation}
\int \mathbb{E}\left(N_1^*\log N_1^*\mid \mathfrak{I}_0\right)d \sf Q_0^*
\end{equation}

To compute the bounds on the expected logarithmic return on wealth, we simply set the logarithmic return on wealth as our $g$; 

To compute the bounds on risk premium that we report in section 6, we extend previous approach as follows:

### 2.1 Bounding Risk Premia

- Set $g(x)=R^w+\zeta R^f$ where $\zeta$ is a "multiplier" that we will search over;


- for alternative $\zeta$, deduce $N_1^*(\zeta)$ and $\sf Q_0^*(\zeta)$ as described in the paper;


- compute:

$$
\log \int \mathbb{E}\left[N_1^*(\zeta)R^w\mid \mathfrak{I}_0\right]d \sf Q_0^*(\zeta) - \log \int \mathbb{E}\left[N_1^*(\zeta)R^f\mid \mathfrak{I}_0\right]d \sf Q_0^*(\zeta)
$$
and minimize with respect to $\zeta$;


- set $g(x)=-R^w-\zeta R^f$, repeat, and use the negative of the minimizer to obtain the upper bound;

Two observations:

i) the objective is not globally convex;

ii) a similar approach may be used to deduce upper and lower bounds on other functions of moments, such as volatility.

## 3. Code Implementation

In [ ]:
# Install packages
try:
    import plotly
except:
    import sys
    !{sys.executable} -m pip install plotly

# Load packages
import time
from source.utilities import *
from source.plotting_module import *

### 3.1 Data
The file “UnitaryData.csv” contains the following data from 1954-2016:

- The first four columns contain Euler equation errors from the unitary risk aversion model corresponding to the 3-month T-bill rate, the market excess return, the SMB excess return, and the HML excess return respectively. Under a feasible belief distortion, all four of these variables should have expectation of zero (conditional or unconditional).


- The column “d.p” contains the dividend-price ratio for the CRSP value-weighted index, computed at the start of the return period. Hence functions of d.p[i] (i.e. quantile indicator functions) are valid instruments for the returns in row i.


- The final column “log.RW” contains values of the logarithmic return on CRSP value-weighted index. We use this as a proxy for the logarithmic return on wealth. This is the random variable who’s expectation we are intersted in bounding.

All returns are quarterly and inflation adjusted.

In [ ]:
# Load data
data = pd.read_csv('data/UnitaryData.csv')
# Show statistics of the data
data.describe()

Given our direct use of dividend-price measures, we purposefully choose a very coarse conditioning of information and split the dividend price ratios into three bins using the three empirical terciles. We take the dividend-price terciles to be a three-state Markov process. Then we multiply each of the first four columns by each of the three columns of the indicator function of dividend-price terciles to form a 12-dimensional $f$. 

When bounding the expected logarithmic return on wealth, we use the last column log.RW as our $g$; 

When bounding the risk premium, we define $g$ as discussed above in section 2.1. 

### 3.2 Computational Strategy

In our application, we just have 3 distinct states, so we can represent the function $e(\cdot)$ as a 3-dimensional vector. Additionally, we are free to impose the normalization $e_1=1$. We can solve the dual problem numerically by something analogous to value function iteration for $e=(1,e_2,e_3)$. Here is the iteration scheme:

1\. Guess $e=(1,1,1)$.

2\. For $k \in \{1,2,3\}$, solve
\begin{equation}
v_k = \min_{\lambda_0} \hat{\mathbb{E}}\left(\exp \left[-\frac{1}{\xi}g(X_1) + \lambda_0f(X_1)\right]e(X_1)\mid \text{state today = } k\right)
\end{equation}

3\. Store
\begin{align*}
\hat{e} &= v/v_1 \\
\hat{\epsilon} &= v_1 \\
\text{error} &= \|\hat{e}-e\|
\end{align*}

4\. Set $e = \hat{e}$.

5\. Iterate steps 2-4 until error is arbitrarily close to zero.

Once we have (approximately) stationary values for $\epsilon^*$ and $e^*$ as well as the optimizing $\lambda_0^*$, we can form the conditional belief distortion
\begin{equation}
N_1 = \frac{1}{\epsilon^*} \exp \left[-\frac{1}{\xi}g(X_1)+\lambda_0^* \cdot f(X_1)\right]\frac{e^*(X_1)}{e^*(X_0)}
\end{equation}

To obtain the unconditional relative entropy, we need to average across states using the implied stationary distribution coming from the distorted probabilities. Define a $3\times 3$ matrix $\tilde{P}$ by 
$$
\tilde{P}_{i,j} = \hat{\mathbb{E}}\left[N_1 \mathcal{1}\left(\text{state tomorrow = j}\right)\mid \text{state today = i}\right]
$$

We should have that $\tilde{P}$ is a transition probability matrix, so $\tilde{P}\mathcal{1}=\mathcal{1}$. Next, solve for the stationary distribution $\pi\in \mathbb{R}^3$ as the dominant left eigenvector of $\tilde{P}$, i.e.
\begin{equation}
\tilde{\pi}^\prime \tilde{P} = \tilde{\pi}^\prime
\end{equation}

Then, the unconditional relative entropy can be computed as
\begin{equation}
\text{RE}(\xi) = \sum_{k=1}^{3}\hat{\mathbb{E}}\left[N_1\log N_1 \mid \text{state today = k}\right]\cdot \tilde{\pi}_k
\end{equation}

In [ ]:
# Count time
time_start = time.time() 

# Initialize the solver
solver = InterDivConstraint(tol=1e-9,max_iter=1000)

# Define g(X) = log Rw
solver.g = solver.log_Rw

# Find ξ that corresponds to 120% min RE
x_min_RE = 1.2
# ξ_lower = solver.find_ξ(x_min_RE=x_min_RE,lower=True,tol=1e-7,max_iter=100)
# ξ_upper = solver.find_ξ(x_min_RE=x_min_RE,lower=False,tol=1e-7,max_iter=100)
ξ_lower = 0.14301961660385132  # presolved solution, user can uncomment the previous lines to resolve it
ξ_upper = 0.14917802810668945  # presolved solution, user can uncomment the previous lines to resolve it

# Solve models with the chosen ξ
result_lower = solver.iterate(ξ_lower,lower=True)
result_upper = solver.iterate(ξ_upper,lower=False)

# Print out the time spent
time_spent = round(time.time()-time_start,4)

### 3.3 Results

In [ ]:
# Print iteration information
print("--- Iteration Ends ---")
print("%sx min rel entropy" % x_min_RE)
print("Time spent: %s seconds ---" % (np.round(time_spent,2)))

# Print converged parameter results
print("\n")
print("--- Converged values for the lower bound problem ---")
print("ϵ: %s" % np.round(result_lower['ϵ'],2))
print("e: %s" % np.round(result_lower['e'],2))
print("λ: %s" % np.round(result_lower['λ'],2))

print(" ")
print("--- Converged values for the upper bound problem ---")
print("ϵ: %s" % np.round(result_upper['ϵ'],2))
print("e: %s" % np.round(result_upper['e'],2))
print("λ: %s" % np.round(result_upper['λ'],2))

# Print transition probability matrix under the original empirical probability
print("\n")
print("--- Transition Probability Matrix (Original) ---")
print(np.round(result_lower['P'],2))

# Print transition probability matrix under distorted probability, lower bound
print(" ")
print("--- Transition Probability Matrix (Distorted, lower bound problem) ---")
print(np.round(result_lower['P_tilde'],2))

# Print transition probability matrix under distorted probability, upper bound
print(" ")
print("--- Transition Probability Matrix (Distorted, upper bound problem) ---")
print(np.round(result_upper['P_tilde'],2))


# Print stationary distribution under the original empirical probability
print("\n")
print("--- Stationary Distribution (Original) ---")
print(np.round(result_lower['π'],2))

# Print stationary distribution under distorted probability, lower bound
print(" ")
print("--- Stationary Distribution (Distorted, lower bound problem) ---")
print(np.round(result_lower['π_tilde'],2))

# Print stationary distribution under distorted probability, upper bound
print(" ")
print("--- Stationary Distribution (Distorted, upper bound problem) ---")
print(np.round(result_upper['π_tilde'],2))

# Print relative entropy
print("\n")
print("--- Relative Entropy (lower bound problem) ---")
print("E[NlogN|state 1] = %s " % np.round(result_lower['RE_cond'][0],4))
print("E[NlogN|state 2] = %s " % np.round(result_lower['RE_cond'][1],4))
print("E[NlogN|state 3] = %s " % np.round(result_lower['RE_cond'][2],4))
print("E[NlogN]         = %s " % np.round(result_lower['RE'],4))

# Print relative entropy
print(" ")
print("--- Relative Entropy (Upper bound problem) ---")
print("E[NlogN|state 1] = %s " % np.round(result_upper['RE_cond'][0],4))
print("E[NlogN|state 2] = %s " % np.round(result_upper['RE_cond'][1],4))
print("E[NlogN|state 3] = %s " % np.round(result_upper['RE_cond'][2],4))
print("E[NlogN]         = %s " % np.round(result_upper['RE'],4))

# Print conditional moment & bounds
print("\n")
print("--- Moment (Original, annualized, %) ---")
print("E[g(X)|state 1]  = %s " % (np.round(result_lower['moment_cond'][0]*400,2)))
print("E[g(X)|state 2]  = %s " % (np.round(result_lower['moment_cond'][1]*400,2)))
print("E[g(X)|state 3]  = %s " % (np.round(result_lower['moment_cond'][2]*400,2)))
print("E[g(X)]  = %s " % (np.round(result_lower['moment']*400,2)))
print(" ")
print("--- Moment (Lower bound, annualized, %) ---")
print("E[Ng(X)|state 1] = %s " % (np.round(result_lower['moment_bound_cond'][0]*400,2)))
print("E[Ng(X)|state 2] = %s " % (np.round(result_lower['moment_bound_cond'][1]*400,2)))
print("E[Ng(X)|state 3] = %s " % (np.round(result_lower['moment_bound_cond'][2]*400,2)))
print("E[Ng(X)] = %s " % (np.round(result_lower['moment_bound']*400,2)))
print(" ")
print("--- Moment (Upper bound, annualized, %) ---")
print("E[Ng(X)|state 1] = %s " % (np.round(result_upper['moment_bound_cond'][0]*400,2)))
print("E[Ng(X)|state 2] = %s " % (np.round(result_upper['moment_bound_cond'][1]*400,2)))
print("E[Ng(X)|state 3] = %s " % (np.round(result_upper['moment_bound_cond'][2]*400,2)))
print("E[Ng(X)] = %s " % (np.round(result_upper['moment_bound']*400,2)))

In [ ]:
time_start = time.time() 
entropy_moment_bounds()
print("Time spent: %s seconds ---" % (round(time.time()-time_start,4)))

## 4. Table and plots in Section 6

In [ ]:
# Initialize the solver
solver = InterDivConstraint(tol=1e-9,max_iter=1000)
# Define g(X) = log Rw
solver.g = solver.log_Rw
result_min = solver.iterate(ξ=100.,lower=True)

In [ ]:
print('Table 1: Empirical and distorted transition probabilities')
print('-------------------------------------------------------')
print('               empirical             min entropy')
print('-------------------------------------------------------')
print('          ',np.round(result_min['P'][0],2),'      ',np.round(result_min['P_tilde'][0],2))
print('P    =    ',np.round(result_min['P'][1],2),'      ',np.round(result_min['P_tilde'][1],2))
print('          ',np.round(result_min['P'][2],2),'      ',np.round(result_min['P_tilde'][2],2))
print('')
print('π    =    ',np.round(result_min['π'],2),   '      ',np.round(result_min['π_tilde'],2))
print('-------------------------------------------------------')

In [ ]:
# Below we load presolved ξs but users can use solver.find_ξ to resolve them.
# Example: solver.find_ξ(x_min_RE=1.2,lower=True, tol=1e-7,max_iter=100)

time_start = time.time()
result_lower_list = []
result_upper_list = []
for i in range(0,6):
    result_lower_list.append(solver.iterate(ξs_lower[i],lower=True))
    result_upper_list.append(solver.iterate(ξs_upper[i],lower=False))

def f1(percent):
    box_chart(result_min,result_lower_list[int(percent/5)],result_upper_list[int(percent/5)])

print('Figure 1: Expected log market return')
interact(f1, percent=widgets.IntSlider(min=0, max=25, step=5, value=20));
print("Time spent: %s seconds ---" % (round(time.time()-time_start,4)))

In [ ]:
# Calculate risk premium
# Case 1: min RE
risk_premia_min, risk_premia_cond_min, risk_premia_empirical, risk_premia_cond_empirical = risk_premia(-1.,x_min_RE=1.,lower=True,ξ_tol=1e-7)
result_risk_min = {'moment':risk_premia_empirical,'moment_cond':risk_premia_cond_empirical,'moment_bound':risk_premia_min,'moment_bound_cond':risk_premia_cond_min}

# Below we load presolved ζs but users can plot the objective function over ζ to refind the optimal ζs. 

# Case 2: 20% higher RE
time_start = time.time()
result_risk_lower_list = []
result_risk_upper_list = []
for i in range(0,6):
    # lower bound
    risk_premia_lower, risk_premia_cond_lower, _ ,_ = risk_premia(ζs_lower[i],x_min_RE=1.+i*0.05,lower=True,ξ_tol=1e-7)
    result_risk_lower = {'moment_bound':risk_premia_lower,'moment_bound_cond':risk_premia_cond_lower}
    result_risk_lower_list.append(result_risk_lower)
    # upper bound
    risk_premia_upper, risk_premia_cond_upper, _ ,_ = risk_premia(ζs_upper[i],x_min_RE=1.+i*0.05,lower=False,ξ_tol=1e-7)
    result_risk_upper = {'moment_bound':risk_premia_upper,'moment_bound_cond':risk_premia_cond_upper}
    result_risk_upper_list.append(result_risk_upper)
    
def f2(percent):
    box_chart(result_risk_min,result_risk_lower_list[int(percent/5)],result_risk_upper_list[int(percent/5)])
    
print("Figure 2: Proportional risk compensations")
interact(f2, percent=widgets.IntSlider(min=0, max=25, step=5, value=20));
print("Time spent: %s seconds ---" % (round(time.time()-time_start,4)))